# Running Neural Networks

This notebook loads scripts for defining a neural network experiment and running it. Parameters that define an experiment include:
* `model_type`: A descriptive tag for a family of experiments (i.e. GCN, Dense)
* `version`: A descriptive sub-tag for a member of a family of experiments
* `data_dir`: Main storage location for experiment data
* `result_dir`: Main storage location for experiment outputs and evaluations
* `feature_func`: String name of a feature-processing function
* `adjacency_func`: String name of an adjacency-defining function
* `num_per_class`: Number of samples per class
* `hidden_layers`: List of hidden layers to use in experiments
* `num_epochs`

In [ ]:
import sys
sys.path.append('C:/Users/chetai/Documents/Projects/moonGen/')

import pdb
import pickle
import numpy as np

# Neural network structure imports
from scripts.pytorch.run_pytorch_models import *

## Setup Experiment

In [ ]:
exp_dict = {
    'model_type': 'GCN',
    'version': 'v1',
    'data_dir': 'C:/Users/chetai/Desktop/moonboard_data/',
    'result_dir': 'C:/Users/chetai/Desktop/moonboard_results/',
    'feature_func': 'gen_multihotfeatures',
    'adjacency_func': 'gen_adjacency_diag_norm',
    'num_per_class': 100,
    'hidden_layers': [32],
    'num_epochs': 50
}
params = setup_experiment(**exp_dict)

## Run and Evaluate Neural Network

In [ ]:
run_experiment(params)